# Self Learning Loop Steps

1. Periodically checks for new data
2. **Retrains the model with the updated data**, and saves the updated model
3. This script can be scheduled to run at regular intervals.

In [ ]:
import os
import shutil
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define directories
data_dir = "/path/to/your/data"
new_data_dir = "new_labels"
image_size = (224, 224)
model_path = 'damage_classification_model.h5'

# Function to move new labeled data to the main data directory
def integrate_new_data():
    if os.path.exists(new_data_dir):
        for label in os.listdir(new_data_dir):
            label_dir = os.path.join(new_data_dir, label)
            target_dir = os.path.join(data_dir, label)
            if not os.path.exists(target_dir):
                os.makedirs(target_dir)
            for file_name in os.listdir(label_dir):
                shutil.move(os.path.join(label_dir, file_name), target_dir)
        shutil.rmtree(new_data_dir)

# Function to retrain the model with the updated data
def retrain_model():
    # Integrate new data
    integrate_new_data()

    # Data generators
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

    train_generator = datagen.flow_from_directory(
        data_dir,
        target_size=image_size,
        batch_size=32,
        class_mode='categorical',
        subset='training'
    )

    validation_generator = datagen.flow_from_directory(
        data_dir,
        target_size=image_size,
        batch_size=32,
        class_mode='categorical',
        subset='validation'
    )

    # Load existing model
    model = load_model(model_path)

    # Retrain the model
    model.fit(
        train_generator,
        validation_data=validation_generator,
        epochs=10,
        verbose=1
    )

    # Save the updated model
    model.save(model_path)

# Schedule the retraining process (example: run every day)
import schedule
import time

schedule.every().day.at("00:00").do(retrain_model)

while True:
    schedule.run_pending()
    time.sleep(1)
